#### 1. Import all Necessary libraries. You can refer to https://github.com/microsoft/HaDes
#### 2. Use the input pkl file with the specified format and continue with the notebook below for the formatted txt file generation.

In [ ]:
import pickle
with open("/custom/path/to/qwen_webq.pkl", 'rb') as f:
#Example input file which has a list-based format of -> [['Q1','A1'],['Q2','A2'],.......,['Qn','An']] which is loaded and further processed in the HaDes-specific format.
#Qi in each sample of the input is the Question of the webq dataset in the case of "qwen_webq.pkl" and Ai is the original generated LLM answer for each Qi.
    orig_ques_answers = pickle.load(f)

In [ ]:
import spacy
from tqdm import tqdm
nlp_sent = spacy.load("en_core_web_sm")
answers = []
def extract_first_sentence(orig_answer):
    doc = nlp_sent(orig_answer)
    sentences = [sent.text for sent in doc.sents]
    return sentences[0]
for question, answer in tqdm(orig_ques_answers):
    answers.append(extract_first_sentence(answer))

In [ ]:
import stanza
stanza.download('en')     
nlp_ner = stanza.Pipeline(lang='en', processors='tokenize,ner')

In [ ]:
def find_subset_indices(orig, subset):
    orig = orig.replace(",", "").replace(".", "")
    orig_list = orig.split()
    len_orig = len(orig_list)
    len_sub = len(subset)
    for i in range(len_orig - len_sub + 1):
        if orig_list[i:i + len_sub] == subset:
            return list(range(i , i + len_sub ))
    return []

In [ ]:
answers = [answer for question, answer in orig_ques_answers]

In [ ]:
import pickle
with open("/custom/path/to/golden_webq_judge_qwen.pkl", 'rb') as f:
#This pkl file contains the golden 0/1 labels for hallucination.
    golden_hallucination = pickle.load(f)

In [ ]:
import json
json_format = []
for answer, elt in tqdm(zip(answers,golden_hallucination)):
    ner_sent = nlp_ner(answer)
    output_list = [ent.text.replace(",", "").replace(".", "") for sent in ner_sent.sentences for ent in sent.ents]
    replaced_ids = []
    for subset in output_list:
        replaced_ids += find_subset_indices(answer, subset.split())
    json_format.append(json.dumps({"replaced": answer, "replaced_ids": replaced_ids[:2], "hallucination": elt}))

In [ ]:
with open('/custom/path/to/qwen_webq_hades.txt', 'w') as file:
    # Iterate over data and write each instance to the file
    for instance in json_format:
        file.write(instance + '\n')

In [ ]:
count = 0
with open('/custom/path/to/qwen_webq_hades.txt', 'r+') as file:
    for line in file:
        json_format1 = json.loads(line.strip())
        # print(json_format1['replaced_ids'])
        if len(json_format1['replaced_ids']) == 1:
            json_format1['replaced_ids'].append(json_format1['replaced_ids'][0])
        if len(json_format1['replaced_ids']) == 0:
            lst = [0,0]
            json_format1['replaced_ids'] = lst
print(count)

In [ ]:
import json

# Function to modify the replaced_ids
def modify_replaced_ids(data):
    for sample in data:
        replaced_ids = sample['replaced_ids']
        
        if len(replaced_ids) == 0:
            sample['replaced_ids'] = [0, 0]
        elif len(replaced_ids) == 1:
            sample['replaced_ids'].append(replaced_ids[0])

# Read data from a text file
with open('/custom/path/to/qwen_webq_hades.txt', 'r') as file:
    lines = file.readlines()

modified_data = []
for line in lines:
    try:
        # Load the JSON object
        sample = json.loads(line.strip())
        modified_data.append(sample)
    except json.JSONDecodeError:
        print(f"Skipping invalid JSON line: {line.strip()}")

modify_replaced_ids(modified_data)
with open('/custom/path/to/qwen_webq_hades1.txt', 'w') as file:   #Modified file.
    for item in modified_data:
        file.write(json.dumps(item) + '\n')

print("Data processing complete. Modified data saved to output.txt.")

In [ ]:
count = 0
with open('/custom/path/to/qwen_webq_hades1.txt', 'r') as file:
    for line in file:
        json_format2 = json.loads(line.strip())
        if len(json_format2['replaced_ids']) < 2:
            count+=1
print(count)